In [1]:
import os
import yaml
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
def flatten(l):
  return [item for sublist in l for item in sublist]

In [3]:
load_dotenv()
# This gets all env variables starting with RIM_
cf = {}
for e in os.environ:
  if e.startswith('RIM_'):
    e_name = e.replace('RIM_', '').lower()
    cf[e_name] = os.getenv(e)
# This tries to convert port numbers to int
cf = {k: (int(v) if v.isnumeric() else v) for k, v in cf.items()}

In [4]:
mc = MongoClient(
  host=cf['mongo_host'],
  port=cf['mongo_port'],
  username=cf['mongo_user'],
  password=cf['mongo_pass'],
)
mc._timeout = 1
db = mc['items']

In [5]:
with open('./data/item-categories.yaml', 'r') as f:
  categories = flatten(list(yaml.safe_load_all(f)))
categories

[{'_id': 'foodstuff',
  'desc': 'All you can eat or drink',
  'subcats': ['greens', 'meat', 'meals', 'beverage', 'liquors']},
 {'_id': 'greens', 'desc': 'Grows in a garden'},
 {'_id': 'meat', 'desc': 'Says Moooo'},
 {'_id': 'meals', 'desc': 'Is full of ingredients'},
 {'_id': 'beverage', 'desc': 'Is liquid'},
 {'_id': 'liquors', 'desc': 'Makes you dizzy'},
 {'_id': 'weapons',
  'desc': 'Made for killing',
  'subcats': ['melee', 'ranged', 'exotic']},
 {'_id': 'melee',
  'desc': "To look into someone's eyes",
  'subcats': ['pointed', 'blunt', 'edged']},
 {'_id': 'ranged', 'desc': 'For the introverts'},
 {'_id': 'exotic', 'desc': 'Is it a frying pan?'}]

In [6]:
db.drop_collection('categories')
db['categories'].insert_many(categories)

InsertManyResult(['foodstuff', 'greens', 'meat', 'meals', 'beverage', 'liquors', 'weapons', 'melee', 'ranged', 'exotic'], acknowledged=True)

In [7]:
with open('./data/items.yaml', 'r') as f:
  items = flatten(list(yaml.safe_load_all(f)))
items

[{'_id': 'Cucumber',
  'cat': 'greens',
  'desc': 'Makes cats scared',
  'opts': ['weight', 'quality', 'shape']},
 {'_id': 'Tomato',
  'cat': 'greens',
  'desc': 'Is not green',
  'opts': ['weight', 'quality', 'color']},
 {'_id': 'Beef',
  'cat': 'meat',
  'desc': 'Once it said Mooo last time',
  'opts': ['weight', 'quality', 'part']}]

In [8]:
items_sorted = {}
for v in items:
  vc = v.copy()
  del vc['cat']
  try:
    items_sorted[v['cat']].append(vc)
  except:
    items_sorted[v['cat']] = [vc]
items_sorted

{'greens': [{'_id': 'Cucumber',
   'desc': 'Makes cats scared',
   'opts': ['weight', 'quality', 'shape']},
  {'_id': 'Tomato',
   'desc': 'Is not green',
   'opts': ['weight', 'quality', 'color']}],
 'meat': [{'_id': 'Beef',
   'desc': 'Once it said Mooo last time',
   'opts': ['weight', 'quality', 'part']}]}

In [9]:
for k, v in items_sorted.items():
  db.drop_collection(k)
  db[k].insert_many(v)